### Visualizing TTC Subway Delays

Data source: [Toronto Open Data - TTC Subway Delay Data](https://open.toronto.ca/dataset/ttc-subway-delay-data/)

This notebook connects to the opendata api and retrieves the last 12 months of TTC delay. For visualization, we will be looking at delays only related to the subway stations.


In [ ]:
#uncomment these on for full outputs.
#from IPython.core.interactiveshell import InteractiveShell

#InteractiveShell.ast_node_interactivity = "all"
#pd.options.display.max_rows = 4000

In [83]:
import pandas as pd
import requests
import json
 
# Get the dataset metadata by passing package_id to the package_search endpoint
# For example, to retrieve the metadata for this dataset:
 
url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/package_show"
params = { "id": "996cfe8d-fb35-40ce-b569-698d51fc683b"}
package = requests.get(url, params = params).json()
print(package["result"])

# to export json if needed
# packageJson = json.dumps(package)

{'license_title': 'Open Government Licence – Toronto', 'owner_unit': None, 'relationships_as_object': [], 'topics': 'Transportation', 'owner_email': 'Andrew.Hutt@ttc.ca', 'excerpt': 'TTC Subway Delay Data', 'private': False, 'owner_division': 'Toronto Transit Commission', 'num_tags': 5, 'id': '996cfe8d-fb35-40ce-b569-698d51fc683b', 'metadata_created': '2019-07-23T18:12:29.554797', 'refresh_rate': 'Monthly', 'title': 'TTC Subway Delay Data', 'license_url': 'https://open.toronto.ca/open-data-license/', 'state': 'active', 'information_url': None, 'license_id': 'open-government-licence-toronto', 'type': 'dataset', 'resources': [{'cache_last_updated': None, 'package_id': '996cfe8d-fb35-40ce-b569-698d51fc683b', 'datastore_active': False, 'id': 'fece136b-224a-412a-b191-8d31eb00491e', 'size': 20206, 'format': 'XLSX', 'state': 'active', 'hash': '', 'description': '', 'is_preview': False, 'last_modified': '2019-08-15T16:25:10.332408', 'url_type': 'upload', 'mimetype': 'application/vnd.openxmlfor

In [84]:
if (len(package["result"]["resources"]) > 12):
    lastTwelve = package["result"]["resources"][-12:]

In [85]:
df = pd.read_excel(lastTwelve[0]["url"])
for x in lastTwelve[1:]:
    df = df.append(pd.read_excel(x["url"]))

In [86]:
df

,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle
0,2020-10-01,00:00,Thursday,MCCOWAN STATION,ERTC,6,12,S,SRT,3026
1,2020-10-01,00:08,Thursday,LANSDOWNE STATION,EUNT,5,11,W,BD,5021
2,2020-10-01,07:44,Thursday,BAY STATION,SUDP,0,0,E,BD,5167
3,2020-10-01,09:01,Thursday,KENNEDY BD STATION,MUTO,3,6,W,BD,5167
4,2020-10-01,09:08,Thursday,LESLIE STATION,TUSC,0,0,W,SHP,6181
...,...,...,...,...,...,...,...,...,...,...
1428,2021-09-30,23:29,Thursday,VAUGHAN MC STATION,TUNOA,3,6,S,YU,5476
1429,2021-09-30,00:40,Thursday,DUNDAS STATION,MUIS,0,0,NaN,YU,0
1430,2021-09-30,01:46,Thursday,PAPE STATION,SUUT,11,18,W,BD,5259
1431,2021-09-30,16:04,Thursday,BAYVIEW STATION,EUSC,0,0,E,SHP,6176


In [87]:
df.isnull().value_counts()

Date   Time   Day    Station  Code   Min Delay  Min Gap  Bound  Line   Vehicle
False  False  False  False    False  False      False    False  False  False      11016
                                                         True   False  False       3952
                                                                True   False         42
                                                         False  True   False          7
dtype: int64

In [88]:
df.fillna("N/A", inplace=True)

In [89]:
df[df.isnull().any(axis=1)]

,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle


In [90]:
df["Station"].value_counts()

VAUGHAN MC STATION        780
FINCH STATION             731
KENNEDY BD STATION        504
BLOOR STATION             491
WARDEN STATION            485
                         ... 
KIPLING - ISLIINGTON S      1
KILPING STATION             1
GUNN BUILDING               1
SHEPPARD TAIL TRACK #2      1
YONGE-UNIVERSITY / BLO      1
Name: Station, Length: 289, dtype: int64

In [91]:
df["Station"].value_counts()[df["Station"].value_counts()>50]


VAUGHAN MC STATION        780
FINCH STATION             731
KENNEDY BD STATION        504
BLOOR STATION             491
WARDEN STATION            485
                         ... 
ST PATRICK STATION         62
SCARBOROUGH RAPID TRAN     61
DOWNSVIEW PARK STATION     59
QUEEN'S PARK STATION       57
LAWRENCE EAST STATION      56
Name: Station, Length: 79, dtype: int64

In [92]:
df["Day"].value_counts()

Thursday     2347
Monday       2251
Wednesday    2238
Friday       2222
Tuesday      2196
Saturday     1905
Sunday       1858
Name: Day, dtype: int64

In [93]:
df["Code"].value_counts()

SUDP     1853
MUIS     1513
MUPAA     941
MUO       914
MUSC      637
         ... 
TRTC        1
ERTB        1
MUODC       1
SREAS       1
MRIE        1
Name: Code, Length: 171, dtype: int64

### Loading in TTC Subway Delay Codes

In order to make sense of what the code values in the delay tables are, we read in the TTC subway delay codes. This can also be in the URL mentioned above.

In [94]:
# read code csv
df_codes = pd.read_excel("data/ttc-subway-delay-codes.xlsx")

In [95]:
# the resulting dataframe has multiple columns
df_codes

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,SUB RMENU CODE,CODE DESCRIPTION,NaN,NaN,SRT RMENU CODE,CODE DESCRIPTION
1,NaN,1.0,EUAC,Air Conditioning,NaN,1.0,ERAC,Air Conditioning
2,NaN,2.0,EUAL,Alternating Current,NaN,2.0,ERBO,Body
3,NaN,3.0,EUATC,ATC RC&S Equipment,NaN,3.0,ERCD,Consequential Delay (2nd Delay Same Fault)
4,NaN,4.0,EUBK,Brakes,NaN,4.0,ERCO,Couplers
...,...,...,...,...,...,...,...,...
125,NaN,125.0,TUS,Crew Unable to Maintain Schedule,NaN,NaN,NaN,NaN
126,NaN,126.0,TUSC,Operator Overspeeding,NaN,NaN,NaN,NaN
127,NaN,127.0,TUSET,Train Controls Improperly Shut Down,NaN,NaN,NaN,NaN
128,NaN,128.0,TUST,Storm Trains,NaN,NaN,NaN,NaN


In [96]:
df_codes.drop(columns=['Unnamed: 0','Unnamed: 1','Unnamed: 4','Unnamed: 5'], inplace=True)

In [97]:
df_codes

,Unnamed: 2,Unnamed: 3,Unnamed: 6,Unnamed: 7
0,SUB RMENU CODE,CODE DESCRIPTION,SRT RMENU CODE,CODE DESCRIPTION
1,EUAC,Air Conditioning,ERAC,Air Conditioning
2,EUAL,Alternating Current,ERBO,Body
3,EUATC,ATC RC&S Equipment,ERCD,Consequential Delay (2nd Delay Same Fault)
4,EUBK,Brakes,ERCO,Couplers
...,...,...,...,...
125,TUS,Crew Unable to Maintain Schedule,NaN,NaN
126,TUSC,Operator Overspeeding,NaN,NaN
127,TUSET,Train Controls Improperly Shut Down,NaN,NaN
128,TUST,Storm Trains,NaN,NaN


In [98]:
df_left = df_codes[['Unnamed: 2','Unnamed: 3']].rename(columns={"Unnamed: 2": "Code", "Unnamed: 3": "Code Description"})
df_left

,Code,Code Description
0,SUB RMENU CODE,CODE DESCRIPTION
1,EUAC,Air Conditioning
2,EUAL,Alternating Current
3,EUATC,ATC RC&S Equipment
4,EUBK,Brakes
...,...,...
125,TUS,Crew Unable to Maintain Schedule
126,TUSC,Operator Overspeeding
127,TUSET,Train Controls Improperly Shut Down
128,TUST,Storm Trains


In [99]:
df_right = df_codes[['Unnamed: 6','Unnamed: 7']].rename(columns={"Unnamed: 6": "Code", "Unnamed: 7": "Code Description"})
df_right = df_right.dropna()
df_right

,Code,Code Description
0,SRT RMENU CODE,CODE DESCRIPTION
1,ERAC,Air Conditioning
2,ERBO,Body
3,ERCD,Consequential Delay (2nd Delay Same Fault)
4,ERCO,Couplers
...,...,...
67,TRNOA,No Operator Immediately Available
68,TRO,Transportation Department - Other
69,TRSET,Train Controls Improperly Shut Down
70,TRST,Storm Trains


In [100]:
df_mergedCodes = df_left.append(df_right, ignore_index=True)
df_mergedCodes

,Code,Code Description
0,SUB RMENU CODE,CODE DESCRIPTION
1,EUAC,Air Conditioning
2,EUAL,Alternating Current
3,EUATC,ATC RC&S Equipment
4,EUBK,Brakes
...,...,...
197,TRNOA,No Operator Immediately Available
198,TRO,Transportation Department - Other
199,TRSET,Train Controls Improperly Shut Down
200,TRST,Storm Trains


In [101]:
len(df_mergedCodes)

202

In [102]:
# Add code type column
start_sub = df_mergedCodes.index[df_mergedCodes["Code"] == 'SUB RMENU CODE']
end_sub =  df_mergedCodes.index[df_mergedCodes["Code"] == 'SRT RMENU CODE']

df_mergedCodes["Type"] = ['SUB RMENU CODE' if x<130 else "SRT RMENU CODE" for x in range(len(df_mergedCodes))]

In [103]:
df_mergedCodes["Code"].value_counts()

PUTSC    1
EUNEA    1
PUTO     1
EUTM     1
PUSCR    1
        ..
EUHV     1
TRO      1
EUTRD    1
PUTNT    1
ERME     1
Name: Code, Length: 202, dtype: int64

In [104]:
df_mergedCodes.drop(df_mergedCodes.loc[df_mergedCodes["Code"] == "SUB RMENU CODE"].index, inplace=True)
df_mergedCodes.drop(df_mergedCodes.loc[df_mergedCodes["Code"] == "SRT RMENU CODE"].index, inplace=True)

In [105]:
df_mergedCodes

,Code,Code Description,Type
1,EUAC,Air Conditioning,SUB RMENU CODE
2,EUAL,Alternating Current,SUB RMENU CODE
3,EUATC,ATC RC&S Equipment,SUB RMENU CODE
4,EUBK,Brakes,SUB RMENU CODE
5,EUBO,Body,SUB RMENU CODE
...,...,...,...
197,TRNOA,No Operator Immediately Available,SRT RMENU CODE
198,TRO,Transportation Department - Other,SRT RMENU CODE
199,TRSET,Train Controls Improperly Shut Down,SRT RMENU CODE
200,TRST,Storm Trains,SRT RMENU CODE


In [106]:
# Merge error code with last 12 months
df_full = df.merge(df_mergedCodes, on="Code", how="left")

In [107]:
df_full

,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle,Code Description,Type
0,2020-10-01,00:00,Thursday,MCCOWAN STATION,ERTC,6,12,S,SRT,3026,Train Control - VOBC,SRT RMENU CODE
1,2020-10-01,00:08,Thursday,LANSDOWNE STATION,EUNT,5,11,W,BD,5021,Equipment - No Trouble Found,SUB RMENU CODE
2,2020-10-01,07:44,Thursday,BAY STATION,SUDP,0,0,E,BD,5167,Disorderly Patron,SUB RMENU CODE
3,2020-10-01,09:01,Thursday,KENNEDY BD STATION,MUTO,3,6,W,BD,5167,Misc. Transportation Other - Employee Non-Char...,SUB RMENU CODE
4,2020-10-01,09:08,Thursday,LESLIE STATION,TUSC,0,0,W,SHP,6181,Operator Overspeeding,SUB RMENU CODE
...,...,...,...,...,...,...,...,...,...,...,...,...
15012,2021-09-30,23:29,Thursday,VAUGHAN MC STATION,TUNOA,3,6,S,YU,5476,No Operator Immediately Available,SUB RMENU CODE
15013,2021-09-30,00:40,Thursday,DUNDAS STATION,MUIS,0,0,N/A,YU,0,Injured or ill Customer (In Station) - Transpo...,SUB RMENU CODE
15014,2021-09-30,01:46,Thursday,PAPE STATION,SUUT,11,18,W,BD,5259,Unauthorized at Track Level,SUB RMENU CODE
15015,2021-09-30,16:04,Thursday,BAYVIEW STATION,EUSC,0,0,E,SHP,6176,Speed Control Equipment,SUB RMENU CODE


In [108]:
# export to excel
df_full.to_csv("output/12_months_ttc.csv")

### Explore Station Data
Let's explore the station data. There are also some cells where no error code coud be found.

In [109]:
df_station = df_full.copy()

In [121]:
df_station["Station"].value_counts()[df["Station"].value_counts()<60]

DOWNSVIEW PARK STATION    59
QUEEN'S PARK STATION      57
LAWRENCE EAST STATION     56
GLENCAIRN STATION         49
YORK UNIVERSITY STATIO    45
                          ..
KIPLING - ISLIINGTON S     1
KILPING STATION            1
GUNN BUILDING              1
SHEPPARD TAIL TRACK #2     1
YONGE-UNIVERSITY / BLO     1
Name: Station, Length: 213, dtype: int64

In [124]:
df_station["Station"].groupby(df_station["Station"]).head()

0               MCCOWAN STATION
1             LANSDOWNE STATION
2                   BAY STATION
3            KENNEDY BD STATION
4                LESLIE STATION
                  ...          
14857    YONGE/UNIVERSITY/SPADI
14860    YONGE/UNIVERSITY/SPADI
14914    YONGE/UNIVERSITY/SPADI
14918    YONGE/UNIVERSITY/SPADI
14957    KING STATION TO EGLINT
Name: Station, Length: 798, dtype: object

In [150]:
station_list = df_station["Station"].apply(lambda x: x.split()[-1])
station_list.groupby(station_list).count().sort_values()

Station
#2                            1
SHEP                          1
ROSEDALE                      1
R                             1
PIONEER                       1
OSGO                          1
MI                            1
MCCOWA                        1
YONGE/UNIVERSITY-SPADI        1
LIN                           1
L                             1
KIPLING                       1
KEN                           1
ISLINGTON                     1
HOUSE                         1
GEO                           1
GARAGE                        1
FI                            1
SHEPAPRD/YO                   1
ENTRANCE                      1
SHEPPARD-YONGE                1
ST.A                          1
WOO                           1
WARDE                         1
VILLAGE                       1
UNIVERSITY/BLOOR              1
UNION                         1
TRA                           1
TR                            1
TAIL                          1
TAI                           1


From above, we see that there are a few typos, and words other than station. Let's fix all stations words, and drop all other entries as we will only be concerned with the stations themselves.
STATO
STAITON
STATI   
STATIO                      

In [161]:
# Create a dictionary to fix the entries, mark non-stations as NA
station_dict = {"STATO": "STATION", "STATIONS": "STATION", "STAITON": "STATION", "STATI": "STATION", "STATIO": "STATION", "STATION":"STATION"}
station_list2 = station_list.apply(lambda x: "NA" if station_dict.get(x) is None else station_dict.get(x))


In [162]:
station_list2

0        STATION
1        STATION
2        STATION
3        STATION
4        STATION
          ...   
15012    STATION
15013    STATION
15014    STATION
15015    STATION
15016    STATION
Name: Station, Length: 15017, dtype: object

In [163]:
station_list2.groupby(station_list2).count().sort_values()

Station
NA           943
STATION    14074
Name: Station, dtype: int64

0        STATION
1        STATION
2        STATION
3        STATION
4        STATION
          ...   
15012    STATION
15013    STATION
15014    STATION
15015    STATION
15016    STATION
Name: Station, Length: 15017, dtype: object